In [ ]:
import sys 
sys.path.append("..") 
from collections import Counter
from itertools import combinations

import pandas as pd
import matplotlib.pyplot as plt

from utils import data_helper,result_helper

In [ ]:
unlabeled_ds_path='../data/train/unlabeled_data_processed.csv'
result_csv_path='../data/train/unlabeled_data_labeled.csv'
unlabeled_df=pd.read_csv(unlabeled_ds_path,index_col='id')

In [ ]:
class_label_to_class_dict = {0: '家居', 1: '房产', 2: '教育', 3: '时尚', 4: '时政', 5: '科技', 6: '财经', 7: '游戏', 8: '娱乐', 9: '体育'}

shot_key_word_dict=dict()
shot_key_word_dict[7]=['通关', '礼包', '副本', '玩家', '普攻', '血量', '女妖', '玩法', '手游', '金币', '大招', '攻略', '游戏', '加成', '攻击力', '法师'] # 游戏
shot_key_word_dict[8]=['演技', '银幕', '传媒', '执导', '主创', '明星', '剧中', '影院', '片中', '导演', '票房', '出演', '播出', '剧本', '蜘蛛侠', '粉丝', '好莱坞', '该片', '预告片', '电视剧', '制片人', '影片', '上映', '主演', '喜剧', '领衔主演', '特辑', '饰演', '电影节', '影迷', '编剧', '剧组', '电影', '监制', '影业', '影视', '演员', '出品', '观影', '动画', '媒体', '该剧'] # 娱乐
shot_key_word_dict[9]=['女排', '运动员', '奥运会', '体育', 'NBA', '发球', '锦标赛', '参赛', '队员', '男排', '奥运', '羽毛球', '标准杆'] # 体育

In [ ]:
num_words_based=2

sub_df=pd.DataFrame(columns=['cutted_content','class_label'])
for key in shot_key_word_dict.keys():
    words_in_other_keys=[]
    for other_key in shot_key_word_dict.keys():
        if key!=other_key:
            words_in_other_keys.extend(shot_key_word_dict[other_key])
    regex_pat_in_other_keys='|'.join(str(word) for word in words_in_other_keys)
    words_self_combine_list=list(combinations(shot_key_word_dict[key],num_words_based))
    for words in words_self_combine_list:
        # print(word)
        # and: '(?=.*subpartA)(?=.*element1)'
        regex_pat_in_self_key=''.join('(?=.*'+str(word)+')' for word in words)
        filtered_df=unlabeled_df[unlabeled_df['cutted_content'].str.contains(regex_pat_in_self_key,regex=True)]
        filtered_df=filtered_df[~filtered_df['cutted_content'].str.contains(regex_pat_in_other_keys,regex=True)]
        filtered_df['class_label']=class_label_to_class_dict[key]
        filtered_df['by_words']=str(words)

        sub_df=sub_df.append(filtered_df)

print(len(sub_df))
sub_df=sub_df.drop_duplicates(subset=['cutted_content']) # 去除重复的cutted_content
print(len(sub_df))
sub_df.to_csv(result_csv_path,index=False)

In [ ]:
# 类别分布
label_cnts=dict(sub_df['class_label'].value_counts())
plt.bar(label_cnts.keys(),label_cnts.values())
plt.xlabel('class_label')
plt.title('Label bar')
print(label_cnts.keys())
print(sub_df['class_label'].value_counts())